In [ ]:
!pip install yfinance plotly gradio


In [ ]:
import gradio as gr
import yfinance as yf
import plotly.graph_objects as go

# Global list to hold the user portfolio
portfolio = []

# Function to get historical stock data using Yahoo Finance
def get_historical_data(symbol, period='1mo', interval='1d'):
    stock_data = yf.download(symbol, period=period, interval=interval)
    return stock_data

# Function to get the latest stock price
def get_stock_price(symbol):
    stock = yf.Ticker(symbol)
    price = stock.history(period="1d")['Close'].iloc[-1]
    return price

# Function to add a stock to the portfolio
def add_stock(symbol, quantity):
    price = get_stock_price(symbol)
    if price:
        portfolio.append({'symbol': symbol, 'quantity': quantity, 'price': price})
        return f"Added {quantity} shares of {symbol} at ${price:.2f} each."
    else:
        return "Invalid stock symbol or unable to fetch data."

# Function to remove a stock from the portfolio
def remove_stock(symbol):
    global portfolio
    portfolio = [stock for stock in portfolio if stock['symbol'] != symbol]
    return f"Removed all shares of {symbol}."

# Function to view the portfolio and track its total value
def view_portfolio():
    total_value = 0
    portfolio_details = "Portfolio:\n"
    for stock in portfolio:
        total_value += stock['quantity'] * stock['price']
        portfolio_details += f"{stock['symbol']} - {stock['quantity']} shares at ${stock['price']:.2f} each\n"
    portfolio_details += f"\nTotal Value: ${total_value:.2f}"
    return portfolio_details

# Function to visualize stock performance (historical data)
def plot_stock_performance(symbol, period='1mo', interval='1d'):
    data = get_historical_data(symbol, period, interval)
    fig = go.Figure()

    # Plot the closing price of the stock
    fig.add_trace(go.Scatter(x=data.index, y=data['Close'], mode='lines', name='Closing Price'))

    fig.update_layout(title=f'{symbol} Stock Price Performance',
                      xaxis_title='Date',
                      yaxis_title='Price (USD)',
                      template='plotly_dark')

    return fig

# Gradio UI setup
def stock_portfolio_tracker():
    with gr.Blocks() as demo:
        with gr.Column():
            # Add Stock
            gr.Markdown("### Add Stock to Portfolio")
            symbol_input = gr.Textbox(label="Stock Symbol (e.g., AAPL, TSLA)")
            quantity_input = gr.Number(label="Quantity of Shares")
            add_stock_button = gr.Button("Add Stock")
            add_stock_button.click(add_stock, inputs=[symbol_input, quantity_input], outputs=gr.Textbox())

            # Remove Stock
            gr.Markdown("### Remove Stock from Portfolio")
            remove_symbol_input = gr.Textbox(label="Stock Symbol to Remove")
            remove_stock_button = gr.Button("Remove Stock")
            remove_stock_button.click(remove_stock, inputs=remove_symbol_input, outputs=gr.Textbox())

            # View Portfolio
            gr.Markdown("### View Portfolio")
            portfolio_button = gr.Button("Show Portfolio")
            portfolio_button.click(view_portfolio, outputs=gr.Textbox())

            # Plot Stock Performance (graph)
            gr.Markdown("### Stock Performance Graph")
            stock_symbol_input = gr.Textbox(label="Stock Symbol (e.g., AAPL, TSLA)", value="AAPL")
            period_input = gr.Dropdown(label="Period", choices=["1mo", "3mo", "1y", "5y"], value="1mo")
            interval_input = gr.Dropdown(label="Interval", choices=["1d", "5d", "1wk", "1mo"], value="1d")
            plot_button = gr.Button("Plot Performance")
            plot_button.click(plot_stock_performance, inputs=[stock_symbol_input, period_input, interval_input], outputs=gr.Plot())

    return demo

# Launch the Gradio interface
demo = stock_portfolio_tracker()
demo.launch()


Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2963986d350afc837.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
